In [1]:
import transformers
import accelerate
import peft
from PIL import ImageFile,Image
from datasets import load_dataset
from transformers import AutoImageProcessor
import torch
from peft import PeftConfig, PeftModel


print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"PEFT version: {peft.__version__}")

/DATA/anurag_2101ai04/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Transformers version: 4.38.2
Accelerate version: 0.28.0
PEFT version: 0.9.0


In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
model_checkpoint = "/DATA/anurag_2101ai04/Assignment/CV/google/vit-base-patch16-224-in21k-lora-indian_food/"

In [4]:

dataset = load_dataset("imagefolder", data_dir="./indian-foods-80", split="train")

In [5]:
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'aloo_matar'

In [6]:

image_processor = AutoImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

In [7]:
# Take an image from the indian-foods-80 dataset
image_path = "./indian-foods-80/test/aloo_matar/aloo_matar_8.jpg"
image = Image.open(image_path)

In [8]:
# image_processor = AutoImageProcessor.from_pretrained(repo_name)
encoding = image_processor(image.convert("RGB"), return_tensors="pt")

In [13]:
from transformers import AutoModelForImageClassification

inference_model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
with torch.no_grad():
    outputs = inference_model(**encoding)
    logits = outputs.logits

predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", inference_model.config.id2label[predicted_class_idx])

Predicted class: aloo_matar
